In [354]:
import os
import pandas as pd
import numpy as np

In [355]:
df = pd.read_csv('butler.csv')
original = df.copy()
df.drop(list(df.filter(regex = 'RANK')), axis = 1, inplace = True)


In [356]:
df['next_pts'] = df['PTS'].shift(-1)
df['next_asts'] = df['AST'].shift(-1)
df['next_rbs'] = df['REB'].shift(-1)
df['next_fg3'] = df['FG3M'].shift(-1)

In [357]:
df.dtypes[df.dtypes == "object"]
df.drop(["WL", "SEASON_YEAR"], axis=1)

,TEAM_ABBREVIATION,GAME_DATE,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,OPPONENT,HOME,next_pts,next_asts,next_rbs,next_fg3
0,GSW,2022-10-18T00:00:00,32.803333,10,22,0.455,4,13,0.308,9,...,0.750,0.250,0.500,0.500,LAL,1,34.0,4.0,5.0,5.0
1,GSW,2022-10-21T00:00:00,35.916667,10,22,0.455,5,11,0.455,9,...,1.000,0.000,0.800,0.200,DEN,1,33.0,2.0,5.0,7.0
2,GSW,2022-10-23T00:00:00,31.350000,11,22,0.500,7,12,0.583,4,...,0.143,0.857,0.091,0.909,SAC,1,21.0,8.0,7.0,4.0
3,GSW,2022-10-25T00:00:00,29.923333,7,17,0.412,4,9,0.444,3,...,0.750,0.250,0.429,0.571,PHX,0,33.0,9.0,7.0,7.0
4,GSW,2022-10-27T00:00:00,36.935000,13,22,0.591,7,14,0.500,0,...,0.571,0.429,0.385,0.615,MIA,1,31.0,6.0,11.0,3.0
5,GSW,2022-10-29T00:00:00,38.903333,10,22,0.455,3,13,0.231,8,...,1.000,0.000,0.500,0.500,CHA,0,32.0,3.0,6.0,4.0
6,GSW,2022-10-30T00:00:00,34.976667,10,24,0.417,4,14,0.286,8,...,0.500,0.500,0.300,0.700,DET,0,23.0,13.0,13.0,4.0
7,GSW,2022-11-01T00:00:00,34.490000,7,14,0.500,4,10,0.400,5,...,0.500,0.500,0.429,0.571,MIA,0,39.0,9.0,3.0,8.0
8,GSW,2022-11-03T00:00:00,38.346667,13,22,0.591,8,15,0.533,5,...,0.625,0.375,0.462,0.538,ORL,0,47.0,8.0,8.0,7.0
9,GSW,2022-11-07T00:00:00,37.716667,17,24,0.708,7,12,0.583,6,...,0.429,0.571,0.471,0.529,SAC,1,40.0,5.0,4.0,6.0


In [358]:
df['team_code'] = df['TEAM_ABBREVIATION'].astype("category").cat.codes
df['opponent_code'] = df['OPPONENT'].astype("category").cat.codes
fulldf = df.copy()
df = df.dropna()

In [359]:
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import HuberRegressor



In [360]:
rr = Ridge(alpha=1)
svrmod = SVR(kernel="linear")
hr = HuberRegressor()
split = TimeSeriesSplit(n_splits=5)
mor = MultiOutputRegressor(rr)

sfs = SequentialFeatureSelector(rr, n_features_to_select=10, direction="forward", cv=split, n_jobs=4)

In [361]:
removed_cols = ["next_pts", "next_asts", "next_rbs", "next_fg3", "SEASON_YEAR", "TEAM_ABBREVIATION", "OPPONENT", "GAME_DATE", "WL", "team_code"]
selected_cols = df.columns[~df.columns.isin(removed_cols)]
df.to_csv('kcsv.csv', index=False)

In [362]:

scaler = MinMaxScaler()
df.loc[:, selected_cols] = scaler.fit_transform(df[selected_cols])

In [363]:
df.describe()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,HOME,next_pts,next_asts,next_rbs,next_fg3,team_code,opponent_code
count,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,...,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.0,55.000000
mean,0.580221,0.411765,0.548663,0.585502,0.359091,0.453030,0.540215,0.419835,0.421212,0.836527,...,0.440734,0.545204,0.454796,0.472727,29.363636,6.272727,6.090909,4.890909,0.0,0.505844
std,0.195147,0.188752,0.258659,0.194378,0.238211,0.249878,0.228864,0.256230,0.238640,0.281016,...,0.272413,0.228774,0.228774,0.503857,8.490438,2.965571,2.489033,1.901975,0.0,0.278833
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,1.000000,2.000000,0.0,0.000000
25%,0.475883,0.294118,0.352941,0.475836,0.250000,0.250000,0.362595,0.272727,0.250000,0.816500,...,0.291715,0.414718,0.289546,0.000000,23.000000,4.000000,5.000000,4.000000,0.0,0.267857
50%,0.608874,0.411765,0.588235,0.602230,0.375000,0.416667,0.545802,0.454545,0.416667,1.000000,...,0.466744,0.562586,0.437414,0.000000,29.000000,6.000000,6.000000,5.000000,0.0,0.500000
75%,0.702671,0.529412,0.705882,0.728625,0.500000,0.666667,0.681298,0.545455,0.541667,1.000000,...,0.583431,0.710454,0.585282,1.000000,34.000000,8.000000,7.000000,6.000000,0.0,0.767857
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,50.000000,15.000000,13.000000,10.000000,0.0,1.000000


In [364]:
sfs3 = SequentialFeatureSelector(mor, n_features_to_select=20, direction="forward", cv=split, n_jobs=4)

In [365]:
sfs3.fit(df[selected_cols], df[["next_pts", "next_asts", "next_rbs", "next_fg3"]])


SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                          estimator=MultiOutputRegressor(estimator=Ridge(alpha=1)),
                          n_features_to_select=20, n_jobs=4)

In [366]:
predictors = list(selected_cols[sfs3.get_support()])
predictors

['FG3A',
 'FTA',
 'OREB',
 'AST',
 'TD3',
 'AVAILABLE_FLAG',
 'USG_PCT',
 'PCT_FGM',
 'PCT_FTA',
 'PCT_OREB',
 'PCT_PTS',
 'AST_RATIO',
 'OREB_PCT',
 'E_USG_PCT',
 'E_PACE',
 'PCT_PTS_2PT_MR',
 'PCT_PTS_OFF_TOV',
 'PCT_PTS_PAINT',
 'PCT_AST_3PM',
 'PCT_UAST_3PM']

In [367]:
def backtest(data, model, predictors, start=4, step=1):
    allpreds = []
    games = sorted(data["GAME_DATE"].unique())

    for i in range(start, len(games), step):
        current_game = games[i]
        train = data[data["GAME_DATE"] < current_game]
        test = data[data["GAME_DATE"] == current_game]

        model.fit(train[predictors], train[['next_pts', "next_asts", "next_rbs", "next_fg3"]])
        preds = model.predict(test[predictors])
        combined = pd.concat([test[["next_pts", "next_asts", "next_rbs", "next_fg3"]],
                               pd.Series(preds[0][0], index=test.index), 
                               pd.Series(preds[0][1], index=test.index),
                               pd.Series(preds[0][2], index=test.index),
                               pd.Series(preds[0][3], index=test.index),
                               ], axis=1)
        # combined.columns = ["actual", "prediction"]
        allpreds.append(combined)

    return pd.concat(allpreds)

In [368]:
predictions = backtest(df, mor, predictors)



In [369]:
results = pd.Series(mor.predict(df.iloc[[40]][predictors])[0])
results = pd.DataFrame(results).transpose()
results.columns = ["PPTS", "PAST", "PREB", "PFG3M"]
results

,PPTS,PAST,PREB,PFG3M
0,34.024235,4.278508,5.39937,5.217148


In [370]:
original.iloc[[40]][["PTS", "AST", "REB", "FG3M"]]

,PTS,AST,REB,FG3M
40,29,4,7,4


In [371]:
predictions

,next_pts,next_asts,next_rbs,next_fg3,0,1,2,3
4,31.0,6.0,11.0,3.0,29.639361,6.791875,6.391990,5.925103
5,32.0,3.0,6.0,4.0,32.755216,4.544412,6.609169,5.915689
6,23.0,13.0,13.0,4.0,27.570702,5.408850,6.097717,4.707316
7,39.0,9.0,3.0,8.0,33.439180,4.606838,7.542380,5.191771
8,47.0,8.0,8.0,7.0,29.309235,7.627280,7.870170,4.880703
9,40.0,5.0,4.0,6.0,30.343687,7.458816,8.494737,4.456459
10,27.0,4.0,6.0,3.0,30.554195,5.679030,6.810089,4.512302
11,16.0,5.0,5.0,3.0,32.493694,6.358484,6.962284,5.333359
12,50.0,6.0,9.0,7.0,33.436898,5.189461,5.808132,6.029791
13,24.0,10.0,6.0,5.0,28.651464,4.489178,5.124694,4.326200


In [372]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["next_pts"], predictions[0])

77.75457494505419

In [373]:
df["next_pts"].describe()

count    55.000000
mean     29.363636
std       8.490438
min      12.000000
25%      23.000000
50%      29.000000
75%      34.000000
max      50.000000
Name: next_pts, dtype: float64

In [374]:
def player_history(df):
    df = df.sort_values("GAME_DATE")
    df["game_number"] = range(0, df.shape[0])
    df['pts_corr'] = list(df[['game_number', 'PTS']].expanding().corr().loc[(slice(None), "game_number"), "PTS"])
    df['pts_corr'].fillna(0, inplace=True)

    df['pts_diff'] = df['PTS'] / df['PTS'].shift(1)
    df['pts_diff'].fillna(df['pts_diff'].mean(), inplace=True)

    df['pts_diff'][df['pts_diff'] == np.inf] = 1

    return df


In [375]:
df = df.groupby("SEASON_YEAR", group_keys=False).apply(player_history)

In [376]:
# pd.Series(svrmod.coef_, index=predictors).sort_values()

In [377]:
diff = predictions['actual'] - predictions['prediction']
diff

KeyError: 'actual'